In [247]:
%%capture 
!pip install h2o==3.40.0.1

In [248]:
import numpy as np
import pandas as pd
import h2o

In [249]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str

def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
        
    return sequence      

def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


def split_peptides_sequences(df_signalPP: pd.DataFrame) -> pd.DataFrame:
    '''
    Splits the amino acid sequences into individual amino acids for each position.

    Parameters:
    -----------
    df_signalPP: pandas.DataFrame
        A DataFrame containing the amino acid sequences.

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the split amino acid sequences.
    '''
    # Initialize an empty list to store the split sequences.
    peptides_split = []

    # Split each amino acid sequence into individual amino acids.
    for k, v in df_signalPP.iterrows():
        sequence = []
        for seq in v['sequence']:
            sequence.append(seq)
        peptides_split.append(sequence)

    # Convert the list of split sequences into a DataFrame and fill NaN values with '-'.
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides


def signal_peptide_predictor(list_of_probabilities: list, amino_acids: str, 
                            n_peptides: int, 
                            number_of_iterations: int, 
                            best_model: h2o.estimators, 
                            training_column_name:str = 'MM_N_peptide_abundance', 
                            max_len_of_signal_peptides = 30,
                            one_hot_encode:bool = True) -> pd.DataFrame:
    '''
    Predicts the best signal peptides from a given number of iterations.

    Parameters:
    -----------
    list_of_probabilities: list
        A list of probabilities.
    amino_acids: str
        A string containing the amino acids used to generate the peptides.
    n_peptides: int
        The number of peptides to generate.
    number_of_iterations: int
        The number of iterations to run the predictor.
    best_model: h2o.estimators
        The trained model to use for prediction.
    training_column_name : str
        The name of the column that the model has been trained on.
    max_len_of_signal_peptides : int
        The maximum lenght of the signal peptides you want to be generated 

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the predicted signal peptides.
    '''
    # Initialize an empty DataFrame to store the predicted peptides.
    data = pd.DataFrame()
    
    # Generate and predict peptides for each iteration.
    for i in range(0, number_of_iterations):
        # Generate new peptides.
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len=max_len_of_signal_peptides)
        
        # Split the peptides into sequences.
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        #if one_hot_encode: 


        # Convert the DataFrame to an H2OFrame and make the columns categorical.
        df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        for column in df_test.columns:
            if column != training_column_name:
                df_test[column] = df_test[column].asfactor()

        # Make predictions on the test data.
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        # Concatenate the new predictions with the existing DataFrame.
        if len(data) == 0:
            data = new_TO_NATURE_peptides.copy()
        else:
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending=False)
            data = data[0:5000]
    
    return data


def one_hot_encode_aa(df):
    """One-hot encode amino acid sequences in a pandas dataframe"""

    # Define amino acid alphabet
    aa_alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                   'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']

    # Determine maximum sequence length in input dataframe
    max_seq_len = df.applymap(len).max().max()

    # Initialize one-hot encoding dataframe
    encoding_df = pd.DataFrame()

    # Loop over columns in input dataframe and encode each amino acid sequence
    for col in df.columns:
        # Extract amino acid sequences from current column
        seqs = df[col].values

        # Initialize 2D array to hold one-hot encoding for current column
        encoding_matrix = np.zeros((len(seqs), max_seq_len * len(aa_alphabet)))

        # Loop over amino acid sequences and one-hot encode each one
        for i, seq in enumerate(seqs):
            # Get one-hot encoding vector for current amino acid sequence
            encoding_vec = np.zeros((max_seq_len, len(aa_alphabet)))
            for j, aa in enumerate(seq):
                encoding_vec[j, aa_alphabet.index(aa)] = 1

            # Flatten encoding vector and add it to encoding matrix for current column
            encoding_matrix[i, :] = encoding_vec.flatten()

        # Convert encoding matrix to dataframe and add it to one-hot encoding dataframe
        col_encoding_df = pd.DataFrame(encoding_matrix)
        col_encoding_df.columns = [f'{col}_{j}' for j in range(max_seq_len * len(aa_alphabet))]
        encoding_df = pd.concat([encoding_df, col_encoding_df], axis=1)

    return encoding_df


def one_hot_decode_aa(df, max_length = 22):
    # Define amino acid alphabet
    aa_alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                    'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']
    sps = []
    peptide = ''
    cols = list(df.columns)

    for index, row in df.iterrows():
        for col in cols: 
            if int(row[col]) ==1: 
                aa_index = int(col.split('_')[1])
                aa_acid = aa_alphabet[aa_index]
                peptide += aa_acid
                if len(peptide) == max_length: 
                    sps.append(peptide)
                    peptide = ''

    df['decoded_sequence'] = sps

    return df

# Gerating artificial Signal Peptides (SPs)

## 0 Introduction 
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

## 1 Amino acid probability matrix

Lets import our df_pwn that was made in a previous notebook:

In [250]:
df_pwn = pd.read_csv('../data/02_all_signal_peptides/df_pwn_for_signal_peptides_found_in_supernatant.csv', index_col = False)
df_pwn

,A,C,D,E,F,G,H,...,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.070312,0.000000,0.003906,0.003906,0.035156,0.000000,0.062500,...,0.214844,0.031250,0.007812,0.046875,0.015625,0.015625,0.000000
2,0.042969,0.000000,0.003906,0.007812,0.179688,0.058594,0.000000,...,0.066406,0.140625,0.050781,0.066406,0.007812,0.015625,0.000000
3,0.070312,0.000000,0.003906,0.000000,0.097656,0.035156,0.015625,...,0.042969,0.179688,0.085938,0.015625,0.011719,0.019531,0.000000
4,0.109375,0.007812,0.007812,0.007812,0.035156,0.027344,0.015625,...,0.015625,0.167969,0.078125,0.046875,0.019531,0.046875,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.992188
64,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094
65,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094
66,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094


In [251]:
amino_acids = list(df_pwn.columns.values)
amino_acids

['A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y',
 '-']

In [252]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())
list_of_probabilities[1]

[0.0703125,
 0.0,
 0.00390625,
 0.00390625,
 0.03515625,
 0.0,
 0.0625,
 0.03515625,
 0.234375,
 0.078125,
 0.03125,
 0.01171875,
 0.03125,
 0.0703125,
 0.21484375,
 0.03125,
 0.0078125,
 0.046875,
 0.015625,
 0.015625,
 0.0]

## 2 Load best model from AutoML

In [253]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,6 hours 58 mins
H2O_cluster_timezone:,Europe/Copenhagen
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,2 months and 16 days
H2O_cluster_name:,H2O_from_python_lucaslevassor_kt1oj7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.574 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [254]:
#h2o.cluster().shutdown()



In [255]:
best_model = h2o.load_model("../data/04_ML_models/0303_m22_only_DeepLearning_grid_2_AutoML_2_20230303_104051_model_1.model")
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_2_AutoML_2_20230303_104051_model_1


Status of Neuron Layers: predicting abundance, regression, gaussian distribution, Quadratic loss, 53,301 weights/biases, 633.9 KB, 130,711 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight           weight_rms           mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  --------------------  ----------  --------------------  -------------------  --------------------  -----------------------
    1        430      Input             15.0
    2        100      RectifierDropout  10.0       0.0   0.0   0.23276296097730462    0.24834179878234863   0.0         0.002958147068917464  0.06376025080680847  0.7357743606181819    0.5684988498687744
    3        100      RectifierDropout  10.0       0.0   0.0   0.0244456773470818     0.0633186399936676    0.0         -0.01655781807404842  0.10062646865844727  0.9424583473571407    0.03104504942893982
    4        1        Linear                       0.0   0.0   0.0038239761841441576  0.004199042916297913  0.0         0.009880792832700536  0.10190147161483765  -0.02711407191228376  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0014377220301334513
RMSE: 0.03791730515389314
MAE: 0.003958906076936472
RMSLE: 0.026782781252380764
Mean Residual Deviance: 0.0014377220301334513

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.001510796507738417
RMSE: 0.03886896586916633
MAE: 0.004752945898345494
RMSLE: 0.027842336754589574
Mean Residual Deviance: 0.001510796507738417

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00486689  0.00492425  0.0166937     0.0101358     0.00536887    0.00189542    0.00251339    0.00316545    0.00139557    0.00375212    0.00284392    0.000904686
mean_residual_deviance  0.0015011   0.00410162  0.0130972     0.00153691    0.000189284   5.45361e-06   7.38957e-05   5.13562e-05   1.15861e-05   1.88382e-05   1.17566e-05   1.4745e-05
mse                     0.0015011   0.00410162  0.0130972     0.00153691    0.000189284   5.45361e-06   7.38957e-05   5.13562e-05   1.15861e-05   1.88382e-05   1.17566e-05   1.4745e-05
r2                      -0.166128   0.246466    0.0166725     0.00115026    0.0342594     -0.223583     -0.02891      0.0428047     -0.304447     -0.66493      -0.480489     -0.0538054
residual_deviance       0.0015011   0.00410162  0.0130972     0.00153691    0.000189284   5.45361e-06   7.38957e-05   5.13562e-05   1.15861e-05   1.88382e-05   1.17566e-05   1.4745e-05
rmse                    0.0200515   0.034945    0.114443      0.0392034     0.0137581     0.0023353     0.00859626    0.00716632    0.00340384    0.0043403     0.00342879    0.00383992
rmsle                   0.0158171   0.0240436   0.0788262     0.0337923     0.0130275     0.00232482    0.00832303    0.00697894    0.0033801     0.00431492    0.00341091    0.00379239

Scoring History: 
    timestamp            duration          training_speed    epochs    iterations    samples    training_rmse    training_deviance    training_mae    training_r2
--  -------------------  ----------------  ----------------  --------  ------------  ---------  ---------------  -------------------  --------------  -------------
    2023-03-03 11:27:22  0.000 sec                           0         0             0          nan              nan    

## 3 Signal peptide predictor algorithm

The signal peptide predictor algorithm works by: 

    1. Generating artificial signal peptides - based on the PWM.
    2. It splits the artificial signal peptides into a dataframe adding '_' to the positions that are empty. 
    3. Initiates the H2O dataframe and makes all values except the feature column categorical
    4. Predicts the columns - based on a previously made ML model
    5. Makes a new dataframe with the best predictions, sorts it and slices out top100 
    6. The loop will continue until the specified number is reached. 
 

In [256]:
%%capture
run = False

if run: 
    best_predicted_SPs = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids, n_peptides =  5000,  
                                                        number_of_iterations = 100, 
                                                        best_model = best_model, 
                                                        max_len_of_signal_peptides=22,
                                                        training_column_name = 'MM_N_peptide_abundance')

In [257]:
#best_predicted_SPs.to_csv('../data/05_best_signal_peptides/model_22_/100_i/best_signal_peptides_i_100.csv', index=False)


In [258]:
best_predicted_SPs = pd.read_csv('../data/05_best_signal_peptides/model_22_/100_i/best_signal_peptides_i_100.csv', index_col=False)

In [259]:
best_predicted_SPs.head(50)


,0,1,2,3,4,5,6,...,16,17,18,19,20,21,predictions
0,M,K,F,S,A,I,S,...,S,A,G,Q,Q,A,0.032961
1,M,M,F,R,Y,I,F,...,A,T,A,A,V,A,0.032511
2,M,L,F,L,A,A,L,...,A,A,A,L,I,P,0.032439
3,M,K,F,G,A,L,A,...,A,A,S,L,V,T,0.030866
4,M,R,V,S,A,A,L,...,S,A,F,L,T,E,0.030141
5,M,Q,V,A,A,V,S,...,P,A,Y,T,L,A,0.028538
6,M,H,F,S,A,T,S,...,A,A,G,L,P,-,0.028211
7,M,M,P,K,A,V,I,...,A,V,P,A,K,Y,0.026956
8,M,R,F,S,A,M,F,...,S,A,A,L,W,P,0.026609
9,M,A,F,S,A,A,C,...,A,P,G,L,A,S,0.026144


In [291]:
new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, 
                                                        amino_acids, 
                                                        n_peptides=1000, 
                                                        max_len=22)

new_TO_NATURE_peptides_split = split_peptides_sequences(new_TO_NATURE_peptides)
one_hot_peptides = one_hot_encode_aa(new_TO_NATURE_peptides_split)
df_w_decoded_seqs = one_hot_decode_aa(one_hot_peptides)
df_w_decoded_seqs

,sequence,length
0,MLILPLLGAGLLLAFACA----,18
1,MKRRSFCILAALLVVFIAV---,19
2,MFASTIQSPLPSLSSL------,16
3,MMLLWSVTLAATLIQMAA----,18
4,MRVYLPIAAASGVVQ-------,15
...,...,...
987,MKYFASAIGVAGLALMAA----,18
988,MFYVSTLYAAMTALAALM----,18
989,MRLSLTLLLLTFCAF-------,15
990,MRRPFLSFAILTVACSVS----,18


In [262]:
new_TO_NATURE_peptides_split = split_peptides_sequences(new_TO_NATURE_peptides)
new_TO_NATURE_peptides_split


,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21
0,M,L,A,L,S,S,A,...,A,S,A,A,L,A,-
1,M,I,T,F,S,I,A,...,S,L,-,-,-,-,-
2,M,S,L,K,L,C,L,...,I,L,A,S,V,-,-
3,M,I,T,L,S,T,L,...,V,L,A,S,V,-,-
4,M,K,F,Q,H,A,A,...,A,A,P,V,L,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,M,R,S,S,T,I,L,...,T,K,E,-,-,-,-
986,M,V,S,K,K,F,S,...,V,L,L,-,-,-,-
987,M,P,L,F,S,V,P,...,P,A,H,D,-,-,-
988,M,K,I,V,L,A,A,...,A,S,G,-,-,-,-


In [279]:
one_hot_function = one_hot_encode_aa(new_TO_NATURE_peptides_split)

In [280]:
pd.set_option('display.max_columns', 15)


In [281]:
one_hot_function

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,...,21_14,21_15,21_16,21_17,21_18,21_19,21_20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [282]:
df_w_decoded_seqs = one_hot_decode_aa(one_hot_function)
df_w_decoded_seqs

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,...,21_15,21_16,21_17,21_18,21_19,21_20,decoded_sequence
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MLALSSATLLFMSSVASAALA-
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MITFSIALLPAALSSSL-----
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MSLKLCLSLLLLIGVILASV--
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MITLSTLALLLLLALVLASV--
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MKFQHAAVIQALTATAAPVL--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MRSSTILLLILSLAITKE----
986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MVSKKFSLTLSLLSLVLL----
987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MPLFSVPFSATVGALPAHD---
988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MKIVLAAILTALLLTASG----


In [283]:
decode_sequences = list(df_w_decoded_seqs['decoded_sequence'])
ref_seqs =  list(new_TO_NATURE_peptides['sequence'])


count = 0
for seq in decode_sequences: 
    if seq in ref_seqs: 
        count += 1

if count == len(decode_sequences): 
    print('successful')


successful


In [284]:
proteomics_data = pd.read_csv('../data/02_all_signal_peptides/ML_signal_peptides_w_legnth_22_030323.csv')
sps =  list(proteomics_data['sequence'])

sps_w_tail = []
for peptide in sps: 
    if len(peptide) <= 22:
        peptide = add_dunder_tail(peptide, max_lenght = 22)
        
    sps_w_tail.append(peptide)

In [285]:
len(sps_w_tail)

735

In [286]:
proteomics_data['sequence_w_tail'] = sps_w_tail
proteomics_data

,index,sequence,length,abundance,sequence_w_tail
0,0,MMVAWWSLFLYGLQVAAPAL,20,1.000000,MMVAWWSLFLYGLQVAAPAL--
1,2,MLRVSAIFMACLLLATAA,18,0.339312,MLRVSAIFMACLLLATAA----
2,6,MRFSAIFTLGLAGTALATPLVE,22,0.119362,MRFSAIFTLGLAGTALATPLVE
3,7,MMVAWWSLFLYGLQVAAPAL,20,0.017628,MMVAWWSLFLYGLQVAAPAL--
4,8,MHLPTLVTLACMAVSAS,17,0.071338,MHLPTLVTLACMAVSAS-----
...,...,...,...,...,...
730,1056,MQVKLFYTLALWAPILVS,18,0.000000,MQVKLFYTLALWAPILVS----
731,1057,MKSLIWALPFIPLAY,15,0.000000,MKSLIWALPFIPLAY-------
732,1058,MWPTRSLSSLFFLSLALGSPVS,22,0.000000,MWPTRSLSSLFFLSLALGSPVS
733,1059,MLLPRLSSLLCLAGLATMPVAN,22,0.000000,MLLPRLSSLLCLAGLATMPVAN


In [287]:
df_for_encoding = proteomics_data[['sequence_w_tail']]
df_for_encoding.columns = ['sequence']
df_for_encoding

,sequence
0,MMVAWWSLFLYGLQVAAPAL--
1,MLRVSAIFMACLLLATAA----
2,MRFSAIFTLGLAGTALATPLVE
3,MMVAWWSLFLYGLQVAAPAL--
4,MHLPTLVTLACMAVSAS-----
...,...
730,MQVKLFYTLALWAPILVS----
731,MKSLIWALPFIPLAY-------
732,MWPTRSLSSLFFLSLALGSPVS
733,MLLPRLSSLLCLAGLATMPVAN


In [288]:
df_for_encoding = split_peptides_sequences(df_for_encoding)
df_for_encoding

,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21
0,M,M,V,A,W,W,S,...,A,A,P,A,L,-,-
1,M,L,R,V,S,A,I,...,T,A,A,-,-,-,-
2,M,R,F,S,A,I,F,...,L,A,T,P,L,V,E
3,M,M,V,A,W,W,S,...,A,A,P,A,L,-,-
4,M,H,L,P,T,L,V,...,A,S,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,M,Q,V,K,L,F,Y,...,L,V,S,-,-,-,-
731,M,K,S,L,I,W,A,...,-,-,-,-,-,-,-
732,M,W,P,T,R,S,L,...,A,L,G,S,P,V,S
733,M,L,L,P,R,L,S,...,A,T,M,P,V,A,N


In [289]:
sequences_one_hot_encoded =one_hot_encode_aa(df_for_encoding)
sequences_one_hot_encoded


,0_0,0_1,0_2,0_3,0_4,0_5,0_6,...,21_14,21_15,21_16,21_17,21_18,21_19,21_20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0
733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
decode_sequences_try = one_hot_decode_aa(sequences_one_hot_encoded)
decode_sequences_try

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,...,21_15,21_16,21_17,21_18,21_19,21_20,decoded_sequence
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MMVAWWSLFLYGLQVAAPAL--
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MLRVSAIFMACLLLATAA----
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,MRFSAIFTLGLAGTALATPLVE
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MMVAWWSLFLYGLQVAAPAL--
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MHLPTLVTLACMAVSAS-----
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MQVKLFYTLALWAPILVS----
731,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,MKSLIWALPFIPLAY-------
732,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,MWPTRSLSSLFFLSLALGSPVS
733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,MLLPRLSSLLCLAGLATMPVAN


In [ ]:
list_of_aa = []
for i in range(0,len(best_predicted_SPs)):
    #signal_peptide = "".join(best_predicted_SPs.iloc[i][:-1].tolist())
    peptide = best_predicted_SPs.iloc[i][:-1].tolist()

    #peptide_wo_dash = [aa for aa in peptide if aa != '-']
    #if '-' in peptide: 
    list_of_aa.append(peptide)
    
len(list_of_aa)

100

In [ ]:
list_of_aa.append(aa1)

In [ ]:
cols = [str(i) for i in range(0,22)]

In [ ]:
best_predicted_SPs['sequence'] = best_predicted_SPs[cols].agg(''.join, axis=1)
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,predictions,sequence
0,M,K,F,S,A,I,S,V,L,A,...,S,L,S,A,G,Q,Q,A,0.032961,MKFSAISVLACIGTSLSAGQQA
1,M,M,F,R,Y,I,F,L,G,V,...,A,A,A,T,A,A,V,A,0.032511,MMFRYIFLGVLALTAAATAAVA
2,M,L,F,L,A,A,L,T,L,A,...,P,G,A,A,A,L,I,P,0.032439,MLFLAALTLALCGYPGAAALIP
3,M,K,F,G,A,L,A,S,S,G,...,A,L,A,A,S,L,V,T,0.030866,MKFGALASSGAGALALAASLVT
4,M,R,V,S,A,A,L,T,I,G,...,N,Y,S,A,F,L,T,E,0.030141,MRVSAALTIGAAMLNYSAFLTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,H,F,L,A,T,V,S,L,L,...,A,A,V,A,A,L,H,A,0.019090,MHFLATVSLLCLASAAVAALHA
96,M,A,I,Q,A,P,F,S,W,G,...,V,R,A,A,D,L,A,A,0.019026,MAIQAPFSWGGPLAVRAADLAA
97,M,R,F,S,K,A,V,F,S,A,...,S,A,A,A,A,P,I,V,0.018995,MRFSKAVFSAGAGLSAAAAPIV
98,M,K,N,P,S,F,L,P,T,A,...,A,L,A,F,A,Q,N,A,0.018968,MKNPSFLPTACMLIALAFAQNA


In [ ]:
best_predicted_SPs = best_predicted_SPs.replace(to_replace = '-', value ='')
best_predicted_SPs['sequence'] =  [string.replace("-", "") for string in list(best_predicted_SPs['sequence'])]

In [ ]:
lenght_of_signal_peptides = []
for k,v in best_predicted_SPs.iterrows(): 
    #print(v['sequence'])
    lenght_of_signal_peptides.append(len(v['sequence']))
best_predicted_SPs['length'] = lenght_of_signal_peptides

In [ ]:
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,predictions,sequence,length
0,M,K,F,S,A,I,S,V,L,A,...,L,S,A,G,Q,Q,A,0.032961,MKFSAISVLACIGTSLSAGQQA,22
1,M,M,F,R,Y,I,F,L,G,V,...,A,A,T,A,A,V,A,0.032511,MMFRYIFLGVLALTAAATAAVA,22
2,M,L,F,L,A,A,L,T,L,A,...,G,A,A,A,L,I,P,0.032439,MLFLAALTLALCGYPGAAALIP,22
3,M,K,F,G,A,L,A,S,S,G,...,L,A,A,S,L,V,T,0.030866,MKFGALASSGAGALALAASLVT,22
4,M,R,V,S,A,A,L,T,I,G,...,Y,S,A,F,L,T,E,0.030141,MRVSAALTIGAAMLNYSAFLTE,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,H,F,L,A,T,V,S,L,L,...,A,V,A,A,L,H,A,0.019090,MHFLATVSLLCLASAAVAALHA,22
96,M,A,I,Q,A,P,F,S,W,G,...,R,A,A,D,L,A,A,0.019026,MAIQAPFSWGGPLAVRAADLAA,22
97,M,R,F,S,K,A,V,F,S,A,...,A,A,A,A,P,I,V,0.018995,MRFSKAVFSAGAGLSAAAAPIV,22
98,M,K,N,P,S,F,L,P,T,A,...,L,A,F,A,Q,N,A,0.018968,MKNPSFLPTACMLIALAFAQNA,22


In [ ]:
SPs_22 = best_predicted_SPs[(best_predicted_SPs[['length']]<23).all(axis=1)]
SPs_22

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,predictions,sequence,length
0,M,L,R,F,S,A,S,P,L,G,...,,,,,,,,0.194617,MLRFSASPLGYKGQAAAADQ,20
1,M,M,R,F,S,W,I,A,F,L,...,,,,,,,,0.185066,MMRFSWIAFLSLLVATAP,18
2,M,V,A,A,S,A,S,F,L,I,...,,,,,,,,0.183458,MVAASASFLILSLVAYLAAQV,21
3,M,V,R,H,S,A,I,F,W,A,...,,,,,,,,0.183349,MVRHSAIFWAAALQVTLP,18
4,M,L,R,F,S,L,F,F,G,A,...,,,,,,,,0.178592,MLRFSLFFGACILQFHAP,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M,L,R,F,A,L,L,L,F,L,...,,,,,,,,0.142601,MLRFALLLFLCSLLSAAA,18
95,M,V,S,V,I,L,I,G,F,L,...,,,,,,,,0.142263,MVSVILIGFLLGLYSTGATV,20
96,M,M,R,F,S,S,L,L,F,L,...,,,,,,,,0.142099,MMRFSSLLFLNMLLIAPAALI,21
97,M,L,R,F,M,V,S,A,M,A,...,,,,,,,,0.141988,MLRFMVSAMASALYALLASL,20


In [ ]:
#SPs_22.to_csv('../data/05_best_signal_peptides/10_000/best_signal_peptides_i_10_000_w_sps_22.csv', index=False)


In [ ]:
h2o.shutdown()

H2O session _sid_a663 closed.


/var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/ipykernel_1638/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()
